In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np

In [4]:
# Function to calculate p values using statsmodels regression, result similar to df.corr()
def calculate_pvalues(df):

    # Start as True, but will be only on the first loop
    first = True
    
    for c in df.columns:

        # If column values are binary use logit else use linreg
        if df[c].unique().shape == (2,):

            log = sm.Logit(df[c], df.loc[:, df.columns != c], verbose=0).fit(disp=0)

            # If first column df_output = p_values dataframe, else concatenate with df_output
            if first:            
                df_output = pd.DataFrame(index=log.pvalues.index, data=log.pvalues, columns=[c])
            else:
                df_output = pd.concat([df_output, pd.DataFrame(index=log.pvalues.index, data=log.pvalues, columns=[c])], axis=1)
                
        else:

            lin = sm.OLS(df[c], df.loc[:, df.columns != c], verbose=0).fit(disp=0)

            # If first column df_output = p_values dataframe, else concatenate with df_output
            if first:            
                df_output = pd.DataFrame(index=lin.pvalues.index, data=lin.pvalues, columns=[c])
            else:
                df_output = pd.concat([df_output, pd.DataFrame(index=lin.pvalues.index, data=lin.pvalues, columns=[c])], axis=1)

        # Only on first loop
        first = False

    return df_output.round(4).fillna(0)

In [5]:
df = pd.read_csv('test_data_CANDIDATE.csv', index_col=0)
df

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,nar,hc,sk,trf
index,,,,,,,,,,,,,,,,,
0,49,M,10000,130,269.0,0,1,163,0,0.0,2.0,0,2,2,0,1,6797.761892
1,61,F,10000,138,166.0,0,0,125,1,3.6,NaN,1,2,2,1,3,4307.686943
2,46,F,10000,140,311.0,0,1,120,1,1.8,NaN,2,3,2,0,1,4118.077502
3,69,F,10000,140,254.0,0,0,146,0,2.0,1.0,3,3,2,1,0,7170.849469
4,51,F,10000,100,222.0,0,1,143,1,1.2,1.0,0,2,2,1,0,5579.040145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,54,F,10000,125,273.0,0,0,152,0,0.5,0.0,1,2,2,2,0,6293.123474
284,42,F,10000,120,240.0,1,1,194,0,0.8,0.0,0,3,2,0,1,3303.841931
285,67,M,10000,106,223.0,0,1,142,0,0.3,NaN,2,2,2,1,0,3383.029119


In [6]:
list(df.columns)

['age',
 'sex',
 'cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal',
 'nar',
 'hc',
 'sk',
 'trf']

In [7]:
# Del cp since it has no use
del df['cp']

# Replace nan with 0
df.fillna(0, inplace=True)

# Convert to capital letters
df['sex'] = df['sex'].str.upper()

# Replace with binaries
df.replace('M', 1, inplace=True)
df.replace('F', 0, inplace=True)

# Changes the order of the columns, I wanted sex to be the first
df = df[['sex', 'age', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca',
         'thal', 'nar', 'hc', 'sk', 'trf']]
df

,sex,age,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,nar,hc,sk,trf
index,,,,,,,,,,,,,,,,
0,1,49,130,269.0,0,1,163,0,0.0,2.0,0,2,2,0,1,6797.761892
1,0,61,138,166.0,0,0,125,1,3.6,0.0,1,2,2,1,3,4307.686943
2,0,46,140,311.0,0,1,120,1,1.8,0.0,2,3,2,0,1,4118.077502
3,0,69,140,254.0,0,0,146,0,2.0,1.0,3,3,2,1,0,7170.849469
4,0,51,100,222.0,0,1,143,1,1.2,1.0,0,2,2,1,0,5579.040145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0,54,125,273.0,0,0,152,0,0.5,0.0,1,2,2,2,0,6293.123474
284,0,42,120,240.0,1,1,194,0,0.8,0.0,0,3,2,0,1,3303.841931
285,1,67,106,223.0,0,1,142,0,0.3,0.0,2,2,2,1,0,3383.029119


In [12]:
# Changing the names of the columns to something more understandable
mapping = {'trestbps':'blood_pressure', 'chol': 'cholesterol', 'fbs': 'blood_sugar', 
                'restecg': 'ekg_results', 'thalach': 'max_heart_rate', 'nar': 'n_arms', 
                'sk':'skin_colour', 'hc':'hair_colour', 'trf':'time_in_traffic', 'oldpeak':'stress_test', 'slope':'stress_test_slope'}

df = df.rename(columns=mapping)
df

,sex,age,blood_pressure,cholesterol,blood_sugar,ekg_results,max_heart_rate,exang,stress_test,stress_test_slope,ca,thal,n_arms,hair_colour,skin_colour,time_in_traffic
index,,,,,,,,,,,,,,,,
0,1,49,130,269.0,0,1,163,0,0.0,2.0,0,2,2,0,1,6797.761892
1,0,61,138,166.0,0,0,125,1,3.6,0.0,1,2,2,1,3,4307.686943
2,0,46,140,311.0,0,1,120,1,1.8,0.0,2,3,2,0,1,4118.077502
3,0,69,140,254.0,0,0,146,0,2.0,1.0,3,3,2,1,0,7170.849469
4,0,51,100,222.0,0,1,143,1,1.2,1.0,0,2,2,1,0,5579.040145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
283,0,54,125,273.0,0,0,152,0,0.5,0.0,1,2,2,2,0,6293.123474
284,0,42,120,240.0,1,1,194,0,0.8,0.0,0,3,2,0,1,3303.841931
285,1,67,106,223.0,0,1,142,0,0.3,0.0,2,2,2,1,0,3383.029119


In [9]:
# Correlation matrix
df.corr()

,sex,age,blood_pressure,cholesterol,blood_sugar,ekg_results,max_heart_rate,exang,stress_test,stress_test_slope,ca,thal,n_arms,hair_colour,skin_colour,time_in_traffic
sex,1.000000,0.095453,0.057594,0.252696,-0.029891,0.044233,0.034308,-0.127497,-0.106737,-0.023036,-0.122047,-0.203524,-0.117275,0.229155,-0.011688,-0.131944
age,0.095453,1.000000,0.284418,0.223444,0.123899,-0.119609,-0.414447,0.097387,0.207286,-0.052076,0.281664,0.065248,0.037999,0.042152,0.164504,0.076198
blood_pressure,0.057594,0.284418,1.000000,0.076746,0.193263,-0.100753,-0.081598,0.084592,0.223608,-0.027966,0.107589,0.048754,-0.015898,-0.020930,0.100978,-0.011933
cholesterol,0.252696,0.223444,0.076746,1.000000,-0.068221,-0.081122,-0.055191,0.035705,0.067293,-0.070119,0.057442,0.028262,-0.064614,0.011009,0.073066,-0.107111
blood_sugar,-0.029891,0.123899,0.193263,-0.068221,1.000000,-0.061941,0.004902,0.006120,0.015351,-0.065713,0.149279,-0.043033,-0.037483,-0.100526,0.034228,-0.069979
ekg_results,0.044233,-0.119609,-0.100753,-0.081122,-0.061941,1.000000,0.039900,-0.055421,-0.059004,0.013629,-0.071174,-0.002350,0.007543,0.063102,0.026605,-0.034840
max_heart_rate,0.034308,-0.414447,-0.081598,-0.055191,0.004902,0.039900,1.000000,-0.355377,-0.336999,0.260357,-0.218919,-0.088503,0.054368,-0.059688,0.053495,-0.038972
exang,-0.127497,0.097387,0.084592,0.035705,0.006120,-0.055421,-0.355377,1.000000,0.291216,-0.099052,0.114274,0.200123,0.054547,-0.081517,0.033683,0.080445
stress_test,-0.106737,0.207286,0.223608,0.067293,0.015351,-0.059004,-0.336999,0.291216,1.000000,-0.219338,0.208484,0.217130,0.019074,0.039061,0.023421,0.110318
stress_test_slope,-0.023036,-0.052076,-0.027966,-0.070119,-0.065713,0.013629,0.260357,-0.099052,-0.219338,1.000000,-0.056454,-0.117711,0.070152,-0.055757,-0.059812,-0.000878


In [10]:
# P values matrix
calculate_pvalues(df)

,sex,age,blood_pressure,cholesterol,blood_sugar,ekg_results,max_heart_rate,exang,stress_test,stress_test_slope,ca,thal,n_arms,hair_colour,skin_colour,time_in_traffic
age,0.1557,0.0000,0.0000,0.0000,0.4044,0.6668,0.0001,0.1243,0.8865,0.2199,0.0002,0.0513,0.0000,0.4549,0.0242,0.0361
blood_pressure,0.4343,0.0000,0.0000,0.5907,0.0653,0.8650,0.0000,0.4701,0.0010,0.8919,0.9900,0.1347,0.0024,0.8354,0.9962,0.7187
cholesterol,0.0002,0.0000,0.5907,0.0000,0.0212,0.3234,0.0108,0.3955,0.4761,0.2563,0.5857,0.2477,0.7901,0.2683,0.6522,0.0524
blood_sugar,0.7921,0.1595,0.0167,0.0352,0.0000,0.4755,0.3641,0.7019,0.6243,0.1314,0.0419,0.2092,0.2705,0.1195,0.7923,0.1054
ekg_results,0.3409,0.6668,0.8650,0.3234,0.3076,0.0000,0.3042,0.6378,0.8197,0.9543,0.7995,0.3503,0.1070,0.4546,0.5234,0.7129
max_heart_rate,0.8438,0.0001,0.0000,0.0108,0.8129,0.3042,0.0000,0.0000,0.0003,0.0004,0.3058,0.0003,0.0000,0.4869,0.1628,0.2042
exang,0.3651,0.3441,0.1227,0.3530,0.7956,0.7927,0.0000,0.0000,0.0120,0.7062,0.8759,0.0038,0.0053,0.1100,0.5468,0.5424
stress_test,0.2390,0.8865,0.0010,0.4761,0.7639,0.8197,0.0003,0.0155,0.0000,0.0188,0.0896,0.0647,0.7049,0.3313,0.7687,0.1924
stress_test_slope,0.4497,0.2199,0.8919,0.2563,0.1377,0.9543,0.0004,0.5288,0.0188,0.0000,0.7671,0.0827,0.9468,0.5829,0.2321,0.9227
ca,0.0417,0.0002,0.9900,0.5857,0.0562,0.7995,0.3058,0.8928,0.0896,0.7671,0.0000,0.1470,0.0940,0.7136,0.6042,0.8260


In [ ]:
# plots a one column heatmap of correlation and p value for all columns

# Creates subplot
fig, axes = plt.subplots(nrows=8, ncols=4, figsize=(20, 40))

# Gets matrices
corr = df.corr()
p_values = calculate_pvalues(df)

# Variable to iter along columns names
c = 0

for i in range(8):
    for j in range(2):
        
        # Gets the current column
        col = df.columns[c]
        c +=1
        
        # Plots correlation matrix for current column x all others
        temp_df = pd.DataFrame(corr.sort_values(by=col, ascending=False)[col]).drop(col)
        sns.heatmap(temp_df, annot=True, ax=axes[i, j * 2], vmin=-1, vmax=1)        
        axes[i, j * 2].set_title('Correlation')
        
        # Plots p value matrix for current column x all others
        temp_df = pd.DataFrame(p_values.sort_values(by=col, ascending=True)[col]).drop(col)
        sns.heatmap(temp_df, annot=True, ax=axes[i, j * 2 + 1], cmap="YlGnBu", vmin=0, vmax=1)
        axes[i, j * 2 + 1].set_title('P Value')

# Space between plots
fig.tight_layout(pad=2.0)